In [ ]:
%%capture
%pip install polars boto3

In [4]:
import polars as pl
import boto3
import os

BUCKET = "minikube-jupyterhub-data"

session = boto3.Session(
    aws_access_key_id = os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name = os.environ["REGION_NAME"]
)

sts = session.client("sts")
token_code = input("Enter token code: ")

response = sts.get_session_token(
    SerialNumber = os.environ["MFA_SERIAL"],
    TokenCode = token_code
)

credentials = response["Credentials"]

aws_access_key_id = credentials["AccessKeyId"]
aws_secret_access_key = credentials["SecretAccessKey"]
aws_session_token = credentials["SessionToken"]
region_name = os.environ["REGION_NAME"]

mfa_session = boto3.Session(
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    aws_session_token = aws_session_token,
    region_name = region_name
)

s3 = mfa_session.client("s3")
print(s3.list_objects(Bucket = BUCKET).keys())

prefix = None
key = "yamls.tgz"
s3_path = f"s3://{BUCKET}/{key}"

labels_schema = {
    "name": pl.String
}

metadata_schema = {
    "name": pl.String,
    "namespace": pl.String,
    "labels": pl.Struct(labels_schema)
}

schema = {
    "apiVersion": pl.String,
    "kind": pl.String,
    "metadata": pl.Struct(metadata_schema)
}

storage_options = {
    "aws_access_key_id": aws_access_key_id,
    "aws_secret_access_key": aws_secret_access_key,
    "aws_session_token": aws_session_token,
    "aws_region": region_name
}

ndjson = pl.scan_ndjson(
    f"s3://{BUCKET}/*.ndjson.gz",
    storage_options = storage_options,
    schema = schema,
    batch_size = 32
).filter(pl.col("kind") == "StorageClass")

ndjson = ndjson.with_columns(pl.col("metadata").name.prefix_fields("metadata.")).unnest("metadata")\
    .with_columns(pl.col("metadata.labels").name.prefix_fields("metadata.labels.")).unnest("metadata.labels")

ndjson.head().collect()

Enter token code:  822582


dict_keys(['ResponseMetadata', 'IsTruncated', 'Marker', 'Contents', 'Name', 'Prefix', 'MaxKeys', 'EncodingType'])


apiVersion,kind,metadata.name,metadata.namespace,metadata.labels.name
str,str,str,str,str
"""storage.k8s.io/v1""","""StorageClass""","""synology-iscsi-storage""",null,null
